<a href="https://colab.research.google.com/github/acycliq/spacetx/blob/master/Demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a Demo of the cell calling algorithm 

---

It should run as is. Just click on 'Runtime` and then select 'Restart and run all'



In [0]:
# GIT_USERNAME = "acycliq"
# GIT_TOKEN = "f2:xxx"
# GIT_REPOSITORY = "spacetx"

In [0]:
# !mkdir "{PROJECT_PATH}"
# !git clone https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git "{PROJECT_PATH}"

In [0]:
from os.path import join
from google.colab import drive
from importlib.machinery import SourceFileLoader
from os.path import join
import logging

In [0]:
logger = logging.getLogger()
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:%(levelname)s:%(message)s"
    )

In [0]:
# Mount the google drive
ROOT = "/content/drive"
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PROJ = "My Drive/Colab Experimental/Python/spacetx" 
PROJECT_PATH = join(ROOT, PROJ)

In [0]:
# Import user modules
common = SourceFileLoader('common', join(PROJECT_PATH, 'src/common.py')).load_module()
utils = SourceFileLoader('utils', join(PROJECT_PATH, 'src/utils.py')).load_module()
geneset = SourceFileLoader('geneset', join(PROJECT_PATH, 'src/geneset.py')).load_module()
config = SourceFileLoader('config', join(PROJECT_PATH, 'src/config.py')).load_module()
gene = SourceFileLoader('gene', join(PROJECT_PATH, 'src/gene.py')).load_module()
cell = SourceFileLoader('cell', join(PROJECT_PATH, 'src/cell.py')).load_module()
klass = SourceFileLoader('klass', join(PROJECT_PATH, 'src/klass.py')).load_module()
spot = SourceFileLoader('spot', join(PROJECT_PATH, 'src/spot.py')).load_module()


In [0]:
# Read the cofiguration file
CONFIG_FILE = join(ROOT, PROJ, 'src/config.yml')
ini = config.read(CONFIG_FILE, 'DEFAULT')

reading CellMap from data/default/CellMap.mat


In [0]:
# make a cell object
cells = cell.Cell(ini)

2019-03-05 18:17:23,553:INFO:Rebasing CellYX to match the one-based indexed Matlab object. 


In [0]:
# make a spots object
spots = spot.Spot(ini)

2019-03-05 18:17:23,575:INFO:********* Getting spotattributes from /content/drive/My Drive/Colab Experimental/Python/spacetx/data_preprocessed/spot_attributes.nc **********


In [0]:
# make a klass object
klasses = klass.Klass(ini['gSet'])

In [0]:
# calc the loglik and populate some of the object's properties
spots.loglik(cells, ini)

2019-03-05 18:17:23,877:INFO:Rebasing SpotYX to match the one-based indexed Matlab object. Not sure this is needed!!
2019-03-05 18:17:23,879:INFO:Rebasing Neighbors to match the one-based indexed Matlab object. Not sure this is needed! REMOVE IT
/content/drive/My Drive/Colab Experimental/Python/spacetx/src/spot.py:90: RuntimeWarning: invalid value encountered in greater
  sanity_check = Neighbors[SpotInCell > 0, 0] + 1 == SpotInCell[SpotInCell > 0]
/content/drive/My Drive/Colab Experimental/Python/spacetx/src/spot.py:93: RuntimeWarning: invalid value encountered in greater
  D[SpotInCell > 0, 0] = D[SpotInCell > 0, 0] + ini['InsideCellBonus']


In [0]:
# make now a genes object
genes = spots.getGenes()

In [0]:
# universe
gSub = ini['gSet'].GeneSubset(genes.names)
gSub = gSub.ScaleCell(0)

In [0]:
# now you can set expressions and logexpressions (as the mean expression over klass)
genes.setKlassExpressions(klasses, ini, gSub)

### This where everything is put together and the algorithm runs until convergenve 

In [0]:
p0 = None
for i in range(ini['CellCallMaxIter']):
    # calc the number of copies of each gene in each cell
    cells.geneCount(spots, genes)
    
    # cell calling: Assign cells to klasses
    cells.klassAssignment(spots, genes, klasses, ini)

    # spot calling: Assign spots to cells
    spots.cellAssignment(cells, genes, klasses)

    # Update parameter
    genes.updateGamma(cells, spots, klasses, ini)

    converged, delta = utils.isConverged(spots, p0, ini['CellCallTolerance'])
    logger.info('Iteration %d, mean prob change %f' % (i, delta))

    # replace p0 with the latest probabilities
    p0 = spots.neighbors['prob']

    if converged:
        print("Success!!")
        break


print("done")

2019-03-05 18:17:29,246:INFO:Cell 0 is classified as Oligo.4 with prob 0.36290928
2019-03-05 18:17:29,248:INFO:cell ---> klass probabilities updated
2019-03-05 18:17:29,920:INFO:spot ---> cell probabilities updated
2019-03-05 18:17:30,087:INFO:Iteration 0, mean prob change 1.000000
2019-03-05 18:17:32,073:INFO:Cell 0 is classified as Oligo.4 with prob 0.39623591
2019-03-05 18:17:32,075:INFO:cell ---> klass probabilities updated
2019-03-05 18:17:32,750:INFO:spot ---> cell probabilities updated
2019-03-05 18:17:32,917:INFO:Iteration 1, mean prob change 0.794103
2019-03-05 18:17:34,841:INFO:Cell 0 is classified as Oligo.4 with prob 0.41011926
2019-03-05 18:17:34,842:INFO:cell ---> klass probabilities updated
2019-03-05 18:17:35,532:INFO:spot ---> cell probabilities updated
2019-03-05 18:17:35,705:INFO:Iteration 2, mean prob change 0.465452
2019-03-05 18:17:37,630:INFO:Cell 0 is classified as Oligo.4 with prob 0.41098491
2019-03-05 18:17:37,632:INFO:cell ---> klass probabilities updated
20

Success!!
done
